In [1]:
import os
import networkx as nx
import random
import numpy as np

In [2]:
flist = ['large_intestine.edgelist', 'large_intestine.edgelist', 'thalamus.edgelist', 'cornea.edgelist', 'spleen.edgelist']
base_dir = '/Users/kyo/Downloads/bio-tissue-networks/'
flist = [os.path.join(base_dir, file) for file in flist]

# Read in source graphs from flist
graphs = [nx.read_edgelist(file,create_using=nx.Graph,nodetype=int) for file in flist]
n_g_in = len(graphs)
g_idx_list = [i for i in range(n_g_in)]

In [20]:
output_sizes = [10,25,50,100,250,500,1000]
n_graphs_per_size = 10

output_sizes = [50]
n_graphs_per_size = 100

num_edges_unweighted = []

# Loop through desired output graph sizes
for n_out in output_sizes:
  # Loop through desired number of graphs per size
  for i in range(n_graphs_per_size):
    # Shuffle list of graph index
    random.shuffle(g_idx_list)

    g_found = False
    for g_idx in g_idx_list:
      g_in = graphs[g_idx]

      # Get size of largest connected component
      largest_cc = max(nx.connected_components(g_in), key=len)

      # Verify that the size of the largest component is >= n_out
      if len(largest_cc) >= n_out:
        g_found = True
        break
    
    if not g_found:
      raise RuntimeError("Could not find graph large enough")
    
    # Reduce g_in to the largest connected component
    g_in = nx.subgraph(g_in, largest_cc)
    
    # Renumber graph
    mapping = {label : i for  i, label in enumerate(list(g_in))}
    g_in = nx.relabel_nodes(g_in, mapping=mapping)

    # Remove any self loops
    g_in.remove_edges_from(nx.selfloop_edges(g_in))

    # Pick initial node
    node_list = list(g_in)
    init_node = random.choice(node_list)

    # Initialize pool to neighbors of 'init_node'
    pool = list(g_in.neighbors(init_node))
    # Add init_node to list of nodes in output graph
    nodes_out = [init_node]

    # While |g_out| < n_out, add new node adjacent to g_out
    while len(nodes_out) < n_out:
      # Pick node from pool
      next_node = random.choice(pool)
      # Add it to output list
      nodes_out.append(next_node)
      # Add new neighbors to pool
      pool = set(pool).union(list(g_in.neighbors(next_node)))
      # Remove any nodes in output from pool
      pool = [x for x in pool if x not in nodes_out]

    # Create output graph 
    g_out = nx.subgraph(g_in, nodes_out)

    num_edges_unweighted.append(g_out.number_of_edges())

    # # Renumber nodes
    # mapping = {label : i for  i, label in enumerate(list(g_out))}
    # g_out = nx.relabel_nodes(g_out, mapping=mapping)

    # Record graph


In [3]:
output_sizes = [10,25,50,100,250,500,1000]
n_graphs_per_size = 10

# Loop through desired output graph sizes
for n_out in output_sizes:
  # Loop through desired number of graphs per size
  for i in range(n_graphs_per_size):
    # Shuffle list of graph index
    random.shuffle(g_idx_list)

    g_found = False
    for g_idx in g_idx_list:
      g_in = graphs[g_idx]

      # Get size of largest connected component
      largest_cc = max(nx.connected_components(g_in), key=len)

      # Verify that the size of the largest component is >= n_out
      if len(largest_cc) >= n_out:
        g_found = True
        break
    
    if not g_found:
      raise RuntimeError("Could not find graph large enough")
    
    # Reduce g_in to the largest connected component
    g_in = nx.subgraph(g_in, largest_cc)
    
    # Renumber graph
    mapping = {label : i for  i, label in enumerate(list(g_in))}
    g_in = nx.relabel_nodes(g_in, mapping=mapping)

    # Remove any self loops
    g_in.remove_edges_from(nx.selfloop_edges(g_in))

    # Pick initial node
    node_list = list(g_in)
    init_node = random.choice(node_list)

    # Initialize pool to neighbors of 'init_node'
    count = [0] * len(node_list)
    weights = [0.0] * len(node_list)
    
    neighbors = list(g_in.neighbors(init_node))
    for n in neighbors:
      count[n] += 1
    weights = [c / sum(count) for c in count]

    # Add init_node to list of nodes in output graph
    nodes_out = [init_node]

    # While |g_out| < n_out, add new node adjacent to g_out
    while len(nodes_out) < n_out:
      # Pick node from pool
      next_node = random.choices(node_list, weights=weights, k=1)[0]
      # Add it to output list
      nodes_out.append(next_node)

      # Get new neighbors
      neighbors = list(g_in.neighbors(next_node))
      for n in neighbors:
        count[n] += 1
      for n in nodes_out:
        count[n] = 0
      weights = [c / sum(count) for c in count]

    # # Create output graph 
    g_out = nx.subgraph(g_in, nodes_out)

    # Renumber nodes
    mapping = {label : i for  i, label in enumerate(list(g_out))}
    g_out = nx.relabel_nodes(g_out, mapping=mapping)

    for u, v in g_out.edges:
      g_out[u][v]['weight'] = 1
    
    # Record graph
    output_name = f'graph_n{n_out}_{i}.tsv'
    nx.write_edgelist(g_out, output_name, delimiter='\t',data=['weight'])
